In [ ]:
import json
from urllib.request import Request, urlopen
import ssl
from pathlib import Path
import shutil

In [ ]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
base_headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0",
}

## Get token

In [ ]:
event_id = "0c6146c7-0a4b-4eda-a716-53cfb33b734f"

In [ ]:
def get_token(event_id, person_id=None):
    req = Request(
        f"https://mysport.photo/public/competitor-service/api/v1/competitors/{event_id}/token",
        headers={"Content-Type": "application/json", **base_headers},
        data=json.dumps({ "personId": person_id } if person_id is not None else {}).encode(),
    )
    res = urlopen(req, context=ctx)
    return json.loads(res.read())["value"]

In [ ]:
token = get_token(event_id)
print(f"{token = }")
headers = { "Authorization": f"Bearer {token}", **base_headers }

## Download registry

In [ ]:
album_id = "0c6146c7-0a4b-4eda-a716-53cfb33b734f"

In [ ]:
def load_registry(album_id, headers, batch_size=100):
    registry = []
    while True:
        url = f"https://mysport.photo/public/photo-service/api/v1/photos?albumId={album_id}&offset={len(registry)}&count={batch_size}"
        req = Request(
            url,
            headers=headers,
        )
        res = urlopen(req, context=ctx)
        data = json.loads(res.read())
        total = data["total"]
        registry += data["values"]
        print(f"registered {len(registry)} of {total}")
        if len(registry) >= total:
            break
    return registry

In [ ]:
registry = load_registry(album_id, headers)

In [ ]:
with open("registry.json", "w") as f:
    f.write(json.dumps(registry))

In [ ]:
with open("registry.json", "r") as f:
    registry = json.loads(f.read())

## Download all photos

In [ ]:
base_dir = Path("photos")
base_dir.mkdir(exist_ok=True)

In [ ]:
for i, item in enumerate(registry):
    path = base_dir / (item["photoId"] + ".jpg")
    if not path.exists():
        with open(path, "wb") as f:
            data = urlopen(Request(item["resources"]["watermark"], headers=headers), context=ctx)
            f.write(data.read())
    print(f"downloaded {i + 1} of {len(registry)}")

## Find person

In [ ]:
person_id = "person12055"
person_token = get_token(event_id, person_id=person_id)
print(f"{token = }")
person_headers = { "Authorization": f"Bearer {person_token}", **base_headers }

In [ ]:
person_registry = load_registry(album_id, headers=person_headers)

In [ ]:
input_dir = Path("input")
input_dir.mkdir(exist_ok=True)

In [ ]:
count = 0
for item in person_registry:
    file_name = item["photoId"] + ".jpg"
    src_path = base_dir / file_name
    dst_path = input_dir / file_name
    if src_path.exists():
        shutil.copy2(src_path, dst_path)
        count += 1
    else:
        print(f"File {src_path} not found")
print(f"Copied {count} of {len(person_registry)} files")